Use LLMs to extract useful information from the Notes in Work Orders, which will be used by RPA programs to achieve business process automation.

In the previous solution, we used a lot of regular expressions and Python programs to implement it. The biggest problem with that solution is that for new patterns in the note, we need to write programs to implement them. In this DEMO, we hope to achieve this by writing new prompts, which allows for rapid development, and even users can do it themselves.·

In [131]:
import os
import json

from langchain.embeddings import QianfanEmbeddingsEndpoint
from langchain.chat_models import QianfanChatEndpoint

from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

print(f"QIANFAN_AK={os.environ['QIANFAN_AK']}")
print(f"QIANFAN_SK={os.environ['QIANFAN_SK']}")

QIANFAN_AK=HCCPsQy5p0Ex1rSEL6oorGQb
QIANFAN_SK=TCVRfPCfbtLr0eDPZRXcywMxcgaNuLDE


## Identify intent (requests)

In [111]:
request_reassign_template  = """You are an expert extraction algorithm to process request reassign request. \
extract the following attributes from the note with json format: 
1. hp \
2. subk \
3. reasons \
4. comments \

Just output json string without any explanation. 
If you do not know the value of an attribute asked \
to extract, return null for the attribute's value. \

Here is a note:
{input}"""


reschedule_template = """You are an expert extraction algorithm to process reschedule request. \
extract the following attributes from the note with json format: 
1. rdt \
2. reasons \
3. comments \

Just output json string without any explanation. \
If you do not know the value of an attribute asked \
to extract, return null for the attribute's value. \

Here is a note:
{input}"""

booking_status_change_template = """You are an expert extraction algorithm to booking status change request. \
extract the following attributes from the note with json format: 
1. status_from \
2. status_to \
3. comments \

Just output json string without any explanation. \
If you do not know the value of an attribute asked \
to extract, return null for the attribute's value. \

Here is a note:
{input}"""


In [112]:
request_reassign_description  = """Good for extracting the notes about request reassign. here are some sample notes:\
1. AMS BOOKING TEMPLATE Request: Request Reassign Reassign: Yes SUBK: BBK Reasons: #CEAvailability Comments: hello, let go \
2. AMS BOOKING TEMPLATE Request: Request Reassign Reassign: Yes SUBK: Reasons: #CEAvailability Comments: please assign this WO to appropriate partner DM Charles Lechleitner. Thanks - CL
"""

reschedule_description = """You are an expert extraction algorithm to process reschedule. here are some sample notes:\
1. AMS FOLLOW UP TEMPLATE Request: Revisit Reschedule: Yes RDT: November 10, 2022 17:00 Comments: change please
"""

booking_status_description = """You are an expert extraction algorithm to booking status change. here are some sample notes:\
1. AMS FOLLOW UP TEMPLATE Request: Dispatch Monitoring Booking Status Change: Yes #sf: Field Phone Support #st: Completed Comments: go
"""


In [113]:
prompt_infos = [
    {
        "name": "request_reassign", 
        "description": request_reassign_description, 
        "prompt_template": request_reassign_template
    },
    {
        "name": "reschedule_template", 
        "description": reschedule_description, 
        "prompt_template": reschedule_template
    },
    {
        "name": "booking_status_change", 
        "description": booking_status_description, 
        "prompt_template": booking_status_change_template
    }
]

In [114]:
llm = QianfanChatEndpoint(model="ERNIE-Bot-4", temperature = 0.01) 

In [115]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
destinations_str

'request_reassign: Good for extracting the notes about request reassign. here are some sample notes:1. AMS BOOKING TEMPLATE Request: Request Reassign Reassign: Yes SUBK: BBK Reasons: #CEAvailability Comments: hello, let go 2. AMS BOOKING TEMPLATE Request: Request Reassign Reassign: Yes SUBK: Reasons: #CEAvailability Comments: please assign this WO to appropriate partner DM Charles Lechleitner. Thanks - CL\n\nreschedule_template: You are an expert extraction algorithm to process reschedule. here are some sample notes:1. AMS FOLLOW UP TEMPLATE Request: Revisit Reschedule: Yes RDT: November 10, 2022 17:00 Comments: change please\n\nbooking_status_change: You are an expert extraction algorithm to booking status change. here are some sample notes:1. AMS FOLLOW UP TEMPLATE Request: Dispatch Monitoring Booking Status Change: Yes #sf: Field Phone Support #st: Completed Comments: go\n'

In [116]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [117]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a note to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [118]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [119]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [135]:
result = chain.run("AMS BOOKING TEMPLATE Request: Request Reassign Reassign: Yes SUBK: Reasons: #CEAvailability Comments: Please send to partner")

json_data = json.loads(result.replace("```json", "").replace("`", ""))
print(json.dumps(json_data, indent=4))



> Entering new MultiPromptChain chain...
request_reassign: {'input': 'AMS BOOKING TEMPLATE Request: Request Reassign Reassign: Yes SUBK: BBK Reasons: #CEAvailability Comments: Please send to partner'}
> Finished chain.
{
    "hp": null,
    "subk": "BBK",
    "reasons": "#CEAvailability",
    "comments": "Please send to partner"
}


In [136]:
result = chain.run("AMS FOLLOW UP TEMPLATE Request: Revisit Reschedule: Yes RDT: November 10, 2022 17:00 Comments: change please")

json_data = json.loads(result.replace("```json", "").replace("`", ""))
print(json.dumps(json_data, indent=4))



> Entering new MultiPromptChain chain...
reschedule_template: {'input': 'AMS FOLLOW UP TEMPLATE Request: Revisit Reschedule: Yes RDT: November 10, 2022 17:00 Comments: change please'}
> Finished chain.
{
    "rdt": "November 10, 2022 17:00",
    "reasons": null,
    "comments": "change please"
}


In [137]:
chain.run("AMS FOLLOW UP TEMPLATE Request: Dispatch Monitoring Booking Status Change: Yes #sf: Field Phone Support #st: Completed Comments: go")

json_data = json.loads(result.replace("```json", "").replace("`", ""))
print(json.dumps(json_data, indent=4))



> Entering new MultiPromptChain chain...
booking_status_change: {'input': 'AMS FOLLOW UP TEMPLATE Request: Dispatch Monitoring Booking Status Change: Yes #sf: Field Phone Support #st: Completed Comments: go'}
> Finished chain.
{
    "rdt": "November 10, 2022 17:00",
    "reasons": null,
    "comments": "change please"
}


In [138]:
print(type(json_data)) 

<class 'dict'>


## UI

In [142]:
import gradio as gr
def extract(input):
    output = chain.run(input)
    return     json.dumps(json.loads(result.replace("```json", "").replace("`", "")), indent=4)
    
gr.close_all()
demo = gr.Interface(fn=extract, inputs="text", outputs="text")
demo.launch(share=False, server_port=int(os.environ['PORT2']))

Closing server running on port: 11920
Running on local URL:  http://127.0.0.1:11920

To create a public link, set `share=True` in `launch()`.




> Entering new MultiPromptChain chain...
booking_status_change: {'input': 'AMS FOLLOW UP TEMPLATE Request: Dispatch Monitoring Booking Status Change: Yes #sf: Field Phone Support #st: Completed Comments: go'}
> Finished chain.


In [140]:
os.environ['PORT1']

'39848'